In [1]:
import shelve
from time import sleep
import vk_api
from vk_api import VkUpload
from vk_api.longpoll import VkLongPoll, VkEventType
import requests
from vk_api.execute import VkFunction
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import json
import datetime
import numpy as np


import settings

In [2]:
start_comm_id = 21000000
comm_N = 25000

In [3]:
vk_func_getMembQuan = VkFunction(args=('x'), code='''
    var memb = [];
    var i = 0;
    while (i < 25) 
    {
        memb.push(API.groups.getMembers({"group_id": i + %(x)s})["count"]);
        i = i + 1;
    };
    return memb;
''')

vk_func_getMembIDList = VkFunction(args=('x', 'y'), code='''
    var offset = %(y)s;
    var IDList = [];
    var i = 0;
    while (i < 25)
    {
        IDList.push(API.groups.getMembers({"group_id": %(x)s, "offset" : offset + i * 1000})["items"]);
        i = i + 1;
    }
    return IDList;
''')

def isCommunityAlive(comm_id):
    try:
        info = vk.wall.get(owner_id=-comm_id, owners_only=1, count=1)
        last_post_date = datetime.datetime.fromtimestamp(info.get('items')[0].get('date'))
        last_count = datetime.datetime.now() - last_post_date
        if (last_count.days < 30):
            return "Alive"#f"Alive, {last_count.days} refresh old"
        else:
            return "Dead"#f"Dead {last_count.days} refresh old"
    except:
        return "No access"

In [4]:
vk_session = vk_api.VkApi(token=settings.MY_COMM_TOKEN) 
try:
    vk_session.auth(reauth=True, token_only=True)
except vk_api.AuthError as error_msg:
    print(error_msg)
    
vk_session.RPS_DELAY = 0.05
vk = vk_session.get_api()

MQData = []
for comm_id in range(start_comm_id, start_comm_id + comm_N, 25):
    MQData.append(vk_func_getMembQuan(vk, comm_id))
    if (comm_id % 1000 == 0):
        print(comm_id - start_comm_id)

Login is required to auth
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000


In [5]:
MQData = [y for x in MQData for y in x]
print(MQData)

[0, 590, 2, 15, 1, 0, 1, 2, 0, 1, 0, 794, 1, 0, 2, 46, 1, 2, 3, None, 0, 0, 11, 1, 7629, 2, 1, 2, 79, 0, 2, 26, None, 3, 0, 1, 4, 1, 19, 0, 24, 4, 0, 1, 0, 0, 0, 1, 8, 2, 3, 4, 14, 0, 1, None, None, 0, 2, 0, 18, None, 2, 0, 1, None, 28, 0, 47, None, 123, 1, 49, 0, None, None, 109, 498, 3, 0, 1, 2, 2, 0, 0, 1, 0, 1, 1, 0, 3, 1, 3, 1, 2, None, 3, 3, 33, 5, 4, None, 2, 1, 52, 3, 112, 297, 1, 2, 1, None, 8, 2, 0, 0, 8, 10, 2, 3, 1, 3, 78, 1, 0, 135, 8, 0, None, 1, 2, 2, 21, 5, 1, 1, 0, 0, 2, 1, None, 1, 2965, 1, 3, 2, 0, 0, 1, 122, 0, 87, 14, 0, 5, 1, 2, 45, 2, 9, 2, 0, 2, 3, 1, 2, 0, 0, 9, 4, 2, 1, None, None, 2, 1, 5, 2, 0, 29, 2, 0, 1, 11, 0, 20, 1141, 6, 2, 1, 0, 0, 1, 4, 0, 2, 3, 1, 0, None, 9, 3, None, 0, 366, 0, 4, 2, 18, 189, 2, 0, 11, 2, 826, 0, 1, 1, 0, 5, 61, 1, 63, 0, 0, 15, 1, 53, 119, 2, 107, None, 0, 10, 612, 3388, 0, 0, 11, 1, 11, 2, 1, 6, None, 3, 1, 0, 1, 1, 2, 1, 11, 1, 1, 0, 296, 93, 1, 3, 0, 50, 4, 1, 9, None, 54, 1, None, 2, 11, 0, 3, 8, 1, 9, 10, 21, 0, 5, 1, 1, 2, 1

In [6]:
MQDict = {}
for i in range(comm_N):
    if (MQData[i] is not None) and (MQData[i] > 1e4):
        MQDict[start_comm_id + i] = MQData[i]
print(MQDict)

{21000731: 12666, 21001418: 25620, 21002017: 17367, 21004202: 22352, 21004707: 13961, 21006385: 25218, 21007679: 29449, 21009160: 27411, 21010004: 39936, 21011756: 13519, 21013155: 39479, 21014430: 15963, 21016711: 11432, 21017037: 24269, 21017258: 84748, 21018000: 19650, 21019768: 18770, 21019877: 34947, 21022222: 68708, 21022786: 22208, 21022845: 10540, 21023593: 15925, 21024423: 54921, 21024851: 23583}


In [7]:
vk_session = vk_api.VkApi(login=settings.MY_USER_LOGIN, password=settings.MY_USER_PASSWORD)
try:
    vk_session.auth(reauth=True, token_only=True)
except vk_api.AuthError as error_msg:
    print(error_msg)
vk = vk_session.get_api()

MQDict = {id : memb for id, memb in MQDict.items() if isCommunityAlive(id) is "Alive"}
print(MQDict)

{21001418: 25620, 21002017: 17367, 21006385: 25218, 21007679: 29449, 21009160: 27411, 21013155: 39479, 21014430: 15963, 21019768: 18770, 21022222: 68708, 21022786: 22208, 21022845: 10540, 21024423: 54921}


In [8]:
IDDict = {}
for id, quan in MQDict.items():
    IDDict[id] = []
    for offset in range(0, quan, 25000):
        IDDict[id].append(vk_func_getMembIDList(vk, x=id, y=offset))
        
for id, ilist in IDDict.items():
    IDDict[id] = [z for x in IDDict[id] for y in x for z in y]

In [9]:
np.save(f'{start_comm_id}to{start_comm_id + comm_N - 1}IDDIct.npy', IDDict) 

In [10]:
for key, val in IDDict.items():
    print(f"{key}: {val[0:10000]}")

21001418: [5264, 37106, 42989, 43724, 46393, 65296, 66182, 120111, 121920, 121938, 122136, 122378, 123808, 124205, 124543, 124667, 124912, 125029, 125364, 125827, 126171, 126204, 127399, 127999, 129980, 139865, 141614, 142464, 163308, 166362, 171328, 177705, 177932, 178298, 178609, 178735, 186499, 192333, 198895, 208241, 208665, 215972, 239915, 241306, 248340, 270978, 275080, 280603, 293244, 293517, 301740, 304007, 307711, 359950, 367798, 370380, 383785, 388983, 396902, 415090, 421799, 425337, 435021, 449781, 470007, 486382, 488334, 508455, 522778, 555095, 555283, 580019, 580324, 589462, 599590, 616775, 625399, 632519, 646319, 647845, 653691, 657220, 661116, 662634, 675067, 690497, 693232, 703972, 717495, 720770, 727269, 734174, 776298, 791839, 794640, 802520, 834499, 835346, 843791, 849594, 850104, 850403, 863077, 890611, 891591, 912508, 926966, 941970, 947763, 1005640, 1006657, 1010484, 1015659, 1023119, 1037453, 1050704, 1052224, 1062124, 1083898, 1104926, 1116231, 1132835, 1148795,